In [17]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as tfl

In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [19]:


x_train = np.load("./cleaned_data/x_train.npy")
y_train = np.load("./cleaned_data/y_train.npy")
train_mu = np.load("./cleaned_data/train_mu.npy")
train_std = np.load("./cleaned_data/train_std.npy")
x_val = np.load("./cleaned_data/x_val.npy")
y_val= np.load("./cleaned_data/y_val.npy")


In [20]:
import keras.backend as K

def my_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    mse = tf.keras.losses.MeanSquaredError()
    return bce(y_true[:,0:-1], y_pred[:,0:-1]) + mse(y_true[:,-1], y_pred[:,-1])

def bce_metric(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_true[:,0:-1], y_pred[:,0:-1], from_logits=False))

def mse_metric(y_true, y_pred):
    return K.mean(K.square(y_pred[:,-1] - y_true[:,-1]), axis=-1)

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true[:,0] * y_pred[:,0], 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true[:,0], 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true[:,0] * y_pred[:,0], 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred[:,0], 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


In [21]:
''' 
Model

how cropping layer works - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-cropping-layers-with-keras.md
'''

' \nModel\n\nhow cropping layer works - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-cropping-layers-with-keras.md\n'

In [22]:
def createModel(input_shape = (23,7)):
    
    X = tfl.Input(input_shape)  # define the input to the model
    flat = tfl.Flatten(input_shape=(23, 7))(X)     # Flatten to pass into linear layers
    d1 = tfl.Dense(1000, activation='relu')(flat)
    d2 = tfl.Dense(500, activation='relu')(d1)
    d3 = tfl.Dense(250, activation='relu')(d2)
    d4 = tfl.Dense(100, activation='relu')(d3)
    d5 = tfl.Dense(3,activation=None)(d4)
    
    # have layer (batch_size, 3). Want to take (b, [0,1]) and turn them into probabilities, and keep (b, [2]) as time
    # https://datascience.stackexchange.com/questions/86740/how-to-slice-an-input-in-keras
    intermediate = tfl.Reshape((3,1), input_shape=(3,))(d5)
    
    probs = tfl.Cropping1D(cropping=(0,1))(intermediate)
    probs = tfl.Activation('softmax')(probs)
    probs = tfl.Reshape((2,), input_shape=(2,1))(probs)
    
    time = tfl.Cropping1D(cropping=(2,0))(intermediate)
    time = tfl.Reshape((1,), input_shape=(1,1))(time)
    
    # concatenate the probabilities and predicted_time_to_sack back into one layer
    out = tfl.Concatenate(axis=-1)([probs, time])
    
    model = Model(inputs=X, outputs=out)        # create model
    
    return model

In [23]:
''' 
Didn't really work
'''

# def createModel(input_shape = (23,7)):
    
#     X = tfl.Input(input_shape)  # define the input to the model
#     flat = tfl.Flatten(input_shape=(23, 7))(X)     # Flatten to pass into linear layers
#     d1 = tfl.Dense(100, activation='relu')(flat)
#     d2 = tfl.Dense(80, activation='relu')(d1)
#     d3 = tfl.Dense(3,activation=None)(d2)
    
#     # have layer (batch_size, 3). Want to take (b, [0,1]) and turn them into probabilities, and keep (b, [2]) as time
#     # https://datascience.stackexchange.com/questions/86740/how-to-slice-an-input-in-keras
#     intermediate = tfl.Reshape((3,1), input_shape=(3,))(d3)
    
#     probs = tfl.Cropping1D(cropping=(0,1))(intermediate)
#     probs = tfl.Activation('softmax')(probs)
#     probs = tfl.Reshape((2,), input_shape=(2,1))(probs)
    
#     time = tfl.Cropping1D(cropping=(2,0))(intermediate)
#     time = tfl.Reshape((1,), input_shape=(1,1))(time)
    
#     # concatenate the probabilities and predicted_time_to_sack back into one layer
#     out = tfl.Concatenate(axis=-1)([probs, time])
    
#     model = Model(inputs=X, outputs=out)        # create model
    
#     return model
    

" \nDidn't really work\n"

In [24]:
model = createModel()

print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 23, 7)]      0           []                               
                                                                                                  
 flatten_1 (Flatten)            (None, 161)          0           ['input_2[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 1000)         162000      ['flatten_1[0][0]']              
                                                                                                  
 dense_4 (Dense)                (None, 500)          500500      ['dense_3[0][0]']                
                                                                                            

In [25]:
opt = Adam(0.005)

model.compile(loss = my_loss, optimizer = opt, metrics = [bce_metric, mse_metric, recall, precision])

In [26]:
y_train[0:3]

array([[ 1. ,  0.3],
       [ 0. , -1. ],
       [ 0. , -1. ]])

In [27]:
NUM_EPOCHS = 5
history = model.fit(x_train[:,:,4:], y_train, epochs=NUM_EPOCHS)

Epoch 1/5
6146/6146 [==============================] - 54s 8ms/step - loss: 15.1321 - bce_metric: 14.0111 - mse_metric: 1.1215 - recall: 0.9323 - precision: 0.0812
Epoch 2/5
6146/6146 [==============================] - 49s 8ms/step - loss: 15.0752 - bce_metric: 14.0111 - mse_metric: 1.0646 - recall: 0.9325 - precision: 0.0812
Epoch 3/5
6146/6146 [==============================] - 61s 10ms/step - loss: 15.0842 - bce_metric: 14.0111 - mse_metric: 1.0742 - recall: 0.9331 - precision: 0.0812
Epoch 4/5
6146/6146 [==============================] - 85s 14ms/step - loss: 15.0842 - bce_metric: 14.0113 - mse_metric: 1.0734 - recall: 0.9313 - precision: 0.0812
Epoch 5/5
6146/6146 [==============================] - 59s 10ms/step - loss: 15.0842 - bce_metric: 14.0113 - mse_metric: 1.0734 - recall: 0.9339 - precision: 0.0812


In [13]:
val_loss, val_bce, val_mse, val_recall, val_precision = model.evaluate(x_val[:,:,4:], y_val, verbose=2)

metrics_df = pd.DataFrame(history.history)


1317/1317 - 2s - loss: 15.3060 - bce_metric: 14.0830 - mse_metric: 1.2230 - recall: 0.1245 - precision: 0.0765 - 2s/epoch - 1ms/step


In [14]:
metrics_df

,loss,bce_metric,mse_metric,recall,precision
0,14.454342,14.011061,0.443690,0.937032,0.081232
1,14.364884,14.011292,0.354240,0.932151,0.081216
2,14.306222,14.011292,0.295580,0.929873,0.081216
3,14.268247,14.011292,0.257634,0.930361,0.081216
4,14.235659,14.011061,0.225058,0.935242,0.081232


In [15]:
print(f"val loss = {val_loss}")
print(f"val_bce = {val_bce}")
print(f"val_mse = {val_mse}")
print(f"val_recall = {val_recall}")
print(f"val_precision = {val_precision}")

val loss = 15.306021690368652
val_bce = 14.083005905151367
val_mse = 1.2229914665222168
val_recall = 0.12452543526887894
val_precision = 0.07647588849067688


In [16]:
# https://www.kdnuggets.com/2021/02/saving-loading-models-tensorflow.html

model_string = f"models/second_model/weights_epochs{NUM_EPOCHS}"
model.save_weights(model_string)